In [18]:
# 1. まず*CP.csv からpair、Edisp、 D3のEint、D3なしのEintの組み合わせを得る
import os
import pandas as pd
from IPython.display import display
import glob

# CSVファイルのパス
# CPファイル
csv_file_path = 'ORIENT_RESULT/SPA_Type_1/M_TS/S-5Me_NH_0@2_cis-keto_90_qst3_csv_904_dh6_pair_9_CP.csv' 
# dim3ファイル
dim3_file_path = "ORIENT_RESULT/SPA_Type_1/M_TS/dim3_pair_9.csv" 

data_dir = "/".join(csv_file_path.split("/")[0:-1])
pair_dir = data_dir + "/PAIR"
os.makedirs(pair_dir, exist_ok=True)

# CSVファイルを読み込む
df = pd.read_csv(csv_file_path)
df['Pair'] = df['File Name'].str.extract(r'pair_(\d+)')

# 'Etotal (D3)'を計算するための関数を定義
def calculate_etotal(row, df, col):
    if row['count'] == 1:
        count_2 = df[(df['Pair'] == row['Pair']) & (df['count'] == 2)][col].values[0]
        count_3 = df[(df['Pair'] == row['Pair']) & (df['count'] == 3)][col].values[0]
        return (row[col] - count_2 - count_3)*2625.5
    return None



# 'Etotal (D3)'を計算して新しいカラムに追加
df['Etotal (D3)'] = df.apply(lambda row: calculate_etotal(row, df, 'SCF Done Value (D3-corrected)'), axis=1)
df['Etotal (B3LYP)'] = df.apply(lambda row: calculate_etotal(row, df, 'SCF Done Value (B3LYP)'), axis=1)

# 必要なカラムのみを選択して新しいDataFrameを作成
result_df = df[['Pair', 'Etotal (D3)', 'Etotal (B3LYP)']].dropna()
result_df["Edisp"] = result_df["Etotal (D3)"] - result_df["Etotal (B3LYP)"] 

# 結果を新しいCSVファイルに保存
output_csv_path = data_dir + '/CP_result_processed.csv'  # 出力先のファイルパスを設定してください
result_df.to_csv(output_csv_path, index=False)


In [19]:
# dim3の結果も必要なものを抽出する
df_dim3 = pd.read_csv(dim3_file_path)
df_dim3['Pair'] = df_dim3['File Name'].str.extract(r'pair_(\d+)')
result_df_dim3 = df_dim3[['Pair', 'Electrostatic Energy', 'Induction Energy']]
dim3_output_csv_path = data_dir + '/CP_result_processed_dim3.csv' 
result_df_dim3.to_csv(dim3_output_csv_path, index=False)

In [20]:
# CSVファイルを読み込む
cp_processed_df = pd.read_csv(output_csv_path)
cp_processed_dim3_df = pd.read_csv(dim3_output_csv_path)

# 'Pair'列を基準にして横方向に結合（concatenate）
concat_df = pd.concat([cp_processed_df.set_index('Pair'), cp_processed_dim3_df.set_index('Pair')], axis=1, join='inner').reset_index()
concat_df["Replusion"] = concat_df["Etotal (B3LYP)"] - concat_df["Electrostatic Energy"] - concat_df["Induction Energy"]
concat_df = concat_df[['Pair', 'Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp', 'Etotal (D3)', 'Etotal (B3LYP)']]

# 結果を新しいCSVファイルに保存
output_csv_path = data_dir + '/processed_DMP_results.csv'  # 出力先のファイルパスを設定してください
concat_df.to_csv(output_csv_path, index=False)

# ソートしたものを保存
sorted_concat_df = concat_df.sort_values(by='Electrostatic Energy')
sorted_concat_df_path = data_dir + '/processed_DMP_results_sorted.csv'
sorted_concat_df.to_csv(sorted_concat_df_path, index=False)

display(sorted_concat_df)

,Pair,Electrostatic Energy,Induction Energy,Replusion,Edisp,Etotal (D3),Etotal (B3LYP)
11,7,-13.954445,-6.042914,18.073039,-43.307766,-45.232085,-1.924320
5,1,-6.924950,-5.992864,14.958439,-36.807415,-34.766790,2.040625
12,8,-5.703423,-1.649162,1.261534,-8.742415,-14.833466,-6.091051
7,3,-3.884637,-2.716936,1.952244,-8.141104,-12.790433,-4.649329
0,10,-2.894799,-1.230868,3.096088,-12.817597,-13.847176,-1.029579
13,9,-2.755492,-0.783983,5.929600,-19.816947,-17.426822,2.390125
9,5,-2.306480,-0.798987,5.873067,-19.232153,-16.464553,2.767600
3,13,-2.000306,-1.155814,5.262906,-14.973083,-12.866297,2.106786
1,11,-1.209981,-8.352886,22.725828,-50.808280,-37.645320,13.162961
4,14,0.123342,-0.485887,4.778630,-11.179171,-6.763086,4.416085


## Groupラベルの追加 (一度目視の確認が必要、上で出力された結果に基づきn_clustersの値を適宜変える)

In [21]:
import pandas as pd
from sklearn.cluster import KMeans

# CSVファイルのパス
csv_file_path = sorted_concat_df_path  

# CSVファイルを読み込む
df = pd.read_csv(csv_file_path)

# KMeansクラスタリングを使用して7つのグループに分ける
kmeans = KMeans(n_clusters=14)
# df['Group'] = kmeans.fit_predict(df[['Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp']])
df['Group'] = kmeans.fit_predict(df[['Electrostatic Energy']])

# Groupカラムを'Electrostatic Energy'の大きい順に1から7まで名付ける
group_means = df.groupby('Group')['Electrostatic Energy'].mean().sort_values(ascending=True)
new_group_mapping = {old: new for new, old in enumerate(group_means.index, start=1)}
df['Group'] = df['Group'].map(new_group_mapping)

df = df[['Pair', 'Group', 'Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp', 'Etotal (D3)', 'Etotal (B3LYP)']]


# 結果を新しいCSVファイルに保存
grouped_csv_path = data_dir + '/processed_DMP_results_sorted_grouped.csv'  
df.to_csv(grouped_csv_path, index=False)

print("等価であることを確認してください")
display(df)

等価であることを確認してください


,Pair,Group,Electrostatic Energy,Induction Energy,Replusion,Edisp,Etotal (D3),Etotal (B3LYP)
0,7,1,-13.954445,-6.042914,18.073039,-43.307766,-45.232085,-1.924320
1,1,2,-6.924950,-5.992864,14.958439,-36.807415,-34.766790,2.040625
2,8,3,-5.703423,-1.649162,1.261534,-8.742415,-14.833466,-6.091051
3,3,4,-3.884637,-2.716936,1.952244,-8.141104,-12.790433,-4.649329
4,10,5,-2.894799,-1.230868,3.096088,-12.817597,-13.847176,-1.029579
5,9,6,-2.755492,-0.783983,5.929600,-19.816947,-17.426822,2.390125
6,5,7,-2.306480,-0.798987,5.873067,-19.232153,-16.464553,2.767600
7,13,8,-2.000306,-1.155814,5.262906,-14.973083,-12.866297,2.106786
8,11,9,-1.209981,-8.352886,22.725828,-50.808280,-37.645320,13.162961
9,14,10,0.123342,-0.485887,4.778630,-11.179171,-6.763086,4.416085


## 中心+group_nのgjfファイル作成 (Groupが等価なことを確認したら)

## PAIRフォルダに"pair_n.gjf"を追加
## PAIRフォルダに"central.gjf"と名付けたファイルを追加、このファイルは中心分子だけの座標を持つファイルでCONSTANT以降の行は消す(改行もすべて)

In [22]:
# 等価なグループのMを持つ行を追加
import shutil
df = pd.read_csv(grouped_csv_path)
groups = df["Group"].unique()
pairs = df["Pair"].unique()
result_path =  pair_dir + "/result"
os.makedirs(result_path, exist_ok=True)
central_file = pair_dir + "/central.gjf"

file_path = pair_dir + "/*.gjf"
paths = [p for p in glob.glob(file_path)]

with open(central_file, 'r') as file:
    lines = file.readlines()

    # 最後の行が空白行でなければ、空白行を追加する
    if lines[-1] != '\n':
        lines.append('\n')

# 変更をファイルに書き戻す
with open(central_file, 'w') as file:
    file.writelines(lines)

for path in paths:
    if 'pair' in path:
        pair_n = int(path.split("pair_")[-1].split(".gjf")[0])
        group_n = df[df["Pair"]==pair_n]["Group"].iloc[-1]
        # コピー先のファイル名を設定
        destination_file = os.path.join(result_path, f'central_+group_{group_n}.gjf')
        # 同じ名前のファイルがすでに存在する場合は処理をスキップ
        if not os.path.exists(destination_file):
            # ファイルをコピー
            shutil.copy(central_file, destination_file)
        else:
            print(f"File '{destination_file}' already exists. Skipping the copy process.")
        
#         pathのfileのM行をdestination_fileの末尾に付け足していく
        with open(path, 'r') as src, open(destination_file, 'a') as dst:
            for line in src:
                if line.endswith(" M\n"):
                    dst.write(line)
